In [1]:
if True:
    import numpy as np
    import pandas as pd

    # Add lab library
    import sys
    sys.path.insert(0, '/home/trevormjs/Documents/Science/APL/Lab')

    #---------------------------------------------------------------------#
    #                        matplotlib plotting                          #
    #---------------------------------------------------------------------#
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    from jupyterthemes import jtplot
    from Helper.plotting import my_graph
    # Edit the font, font size, and axes width
    # mpl.rcParams['font.family'] = 'Avenir'
    plt.rcParams['font.size'] = 24
    plt.rcParams['axes.linewidth'] = 2
    jtplot.style(theme='monokai', context='notebook', ticks=False, grid=False)

    #---------------------------------------------------------------------#
    #                           bokeh plotting                            #
    #---------------------------------------------------------------------#
    from bokeh.plotting import figure, show, output_notebook
    from bokeh.themes import Theme
    from bokeh.io import curdoc, export_png
    from bokeh.models import (Range1d, Label, ColumnDataSource, LabelSet,
                              Legend)
    from Helper.plotting import style
    output_notebook()
    # curdoc().theme = Theme(filename="../Helper/theme.yml")

    #---------------------------------------------------------------------#
    #                       error and unit handling                       #
    #---------------------------------------------------------------------#
    from uncertainties import ufloat
    import Helper.numbers as nu
    from Helper.record import Measurement, Unit

    %load_ext autoreload
    %autoreload 2

Loading BokehJS ...

# Basic FFT

## 1, Sine wave

In [2]:
x = np.linspace(0, .1, 2048)
y = np.sin(2*np.pi*100*x)
window = np.hamming(len(x))
coefs = np.fft.fft(window*y)
freqs = np.fft.fftfreq(y.shape[0], d=x[1]-x[0])

In [3]:
fig = figure()
fig.line(x, y, color = 'blue')
fig.line(x, np.hamming(y.shape[0]), color = 'red')
fig.line(x, y*window, color = 'purple')
# fig.x_range = Range1d(80, 120)
show(fig)

In [4]:
fig = figure()
fig.line(np.fft.fftshift(freqs), np.fft.fftshift(np.abs(coefs)))
fig.x_range = Range1d(80, 120)
show(fig)

In [5]:
nu.halfmax(np.abs(coefs), [coefs.argmax()])[0]*(freqs[1]-freqs[0])

array([17.42189888])

In [6]:
np.fft.fftshift(freqs)[-1]

10225.0048828125

## 2, Larger number of cycles

In [7]:
x = np.linspace(0, 1, 20480)
y = np.sin(2*np.pi*100*x)
window = np.hamming(len(x))
coefs = np.fft.fft(window*y)
freqs = np.fft.fftfreq(y.shape[0], d=x[1]-x[0])

In [8]:
fig = figure()
fig.line(x, y, color = 'blue')
fig.line(x, np.hamming(y.shape[0]), color = 'red')
fig.line(x, y*window, color = 'purple')
# fig.x_range = Range1d(80, 120)
show(fig)

In [9]:
fig = figure()
fig.line(np.fft.fftshift(freqs), np.fft.fftshift(np.abs(coefs)))
fig.x_range = Range1d(80, 120)
show(fig)

In [10]:
nu.halfmax(np.abs(coefs), [coefs.argmax()])[0]*(freqs[1]-freqs[0])

array([1.74206762])

In [11]:
np.fft.fftshift(freqs)[-1]

10238.500048828124

The more cycles of a signal, the higher the certainty of the frequency. This is why
we observe a linewidth of 17 Hz for the 10-cycle signal, and a linewidth of 1.7 for the 100-cycle signal.

The nyquist frequency is defined by the sample rate, which is the same for both signals. However, the number of frequency bins is defined by the number of total samples. Therefore, the range of frequency of the two signals is the same, but the one with more samples is slightly closer to the nyquist frequency due to the higher resolution.

# Sine and Square Waves

In [12]:
def read_scope_csv(path, n_sigs = 1):
    ret = []
    for n in range(n_sigs):
        config = pd.read_csv(path, header=None, usecols=[1+6*n, 2+6*n]).loc[:2]
        config = [record_length, sample_interval, trigger_point] = [
            [float(val), unit] for val, unit in zip(config[1+6*n], config[2+6*n])]
        config= {
            'record_length': record_length,
            'sample_interval': sample_interval,
            'trigger_point': trigger_point
        }
        display(config)
        data = pd.read_csv(path, header = None, usecols=[3+6*n, 4+6*n])
        data.columns = ['ts', 'mV']
        ret.append([data, config])
    return ret

In [13]:
[[sine_wave, sine_wave_params]] = read_scope_csv('./Data/l5_B_a.csv')
sine_wave

sine_wave = sine_wave.loc[:2047]

sine_fft = np.fft.fft(sine_wave.mV * np.hamming(2048), )
freqs = np.fft.fftfreq(2048, sine_wave.ts[1] - sine_wave.ts[0])

fig = figure()
fig.line(np.fft.fftshift(freqs), np.fft.fftshift(np.abs(sine_fft)))
fig.x_range = Range1d(0, 10000)
show(fig)

{'record_length': [2500.0, 'Points'],
 'sample_interval': [2e-06, 's'],
 'trigger_point': [1250.0002, 'Samples']}

In [14]:
[[square_wave, square_wave_params]] = read_scope_csv('./Data/l5_B_b.csv')
square_wave

square_wave = square_wave.loc[:2047]

square_fft = np.fft.fft(square_wave.mV * np.hamming(2048), 2048*16)
freqs = np.fft.fftfreq(2048 * 16, square_wave.ts[1] - square_wave.ts[0])

fig = figure()
fig.line(np.fft.fftshift(freqs), np.fft.fftshift(np.abs(square_fft)))
fig.x_range = Range1d(0, 10000)
show(fig)

{'record_length': [2500.0, 'Points'],
 'sample_interval': [2e-06, 's'],
 'trigger_point': [1250.0002, 'Samples']}

# Tuning Forks